# Automated ML

Importing all needed dependencies to complete the project.

In [ ]:
import logging
import os
import json
import csv
import numpy as np
import pandas as pd
import pkg_resources
import joblib

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core import Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from impyute.imputation.cs import fast_knn
import sys
from sklearn.preprocessing import StandardScaler

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Dataset

### Overview
This dataset comes from the Diabetes and Digestive and Kidney Disease National Institutes. The purpose of this dataset is to diagnose whether or not a patient is diabetic, on the basis of certain diagnostic measures in the dataset. The selection of these instances from a larger database was subject to several restrictions. All patients are women from the Indian heritage of Pima, at least 21 years old. 
https://www.kaggle.com/uciml/pima-indians-diabetes-database

Task
Predict the "Outcome" column based on the input features, either the patient has diabetes or not.

Reference: http://www.arogyaworld.org/wp-content/uploads/2010/10/ArogyaWorld_IndiaDiabetes_FactSheets_CGI2013_web.pdf

The dataset has nine features as follow:
- Pregnancies: Number pregnancy times (int).
- Glucose: Plasma glucose concentration level (int).
- BloodPressure: Diastolic blood pressure level in mm Hg(int).
- SkinThickness: skinfold thickness in mm(int).
- Insulin: two-hour serum insulin measured by mu U/ml(int).
- BMI: Body mass index (float).
- DiabetesPedigreeFunction: Diabetes pedigree function(float).
- Age: age in years 21 and above(int).
- Outcome: Target column 0 or 1, 0 = Not diabetes, 1 = diabetes(int).

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl'

experiment=Experiment(ws, experiment_name)

In [ ]:
# Load the registered dataset from workspace
dataset = Dataset.get_by_name(ws, name='diabetes')

# Convert the dataset to dataframe
df = dataset.to_pandas_dataframe()
df.info()

In [ ]:
df.describe()

In [ ]:
#Display the first five records of the dataset
df.head()

In [ ]:
# Create CPU cluster
amlcompute_cluster_name = "projectcluster"

# Verify if cluster does not exist otherwise use the existing one
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_V2',
                                                           vm_priority = 'lowpriority', 
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


In [ ]:
# Scaling data
scaler = StandardScaler()
scaler.fit(dataset.drop('Outcome', axis=1))
scaler_features = scaler.transform(dataset.drop('Outcome', axis=1))
df_feat = pd.DataFrame(scaler_features, columns=dataset.columns[:-1])

# appending the outcome feature
df_feat['Outcome'] = dataset['Outcome'].astype(int)
dataset_updated = df_feat.copy()

## AutoML Configuration

Overview of the automl settings and configuration used for this experiment:

- "experiment_timeout_minutes": set to 30 minutes. The experiment will timeout after that period to avoid wasting resources.
- "max_concurrent_iterations": is set to 4. The max number of concurrent iterations to be run in parallel at the same time.
- "primary_metric" : is set to 'accuracy', which is a sutible metric for classification problems.
- "n_cross_validations": is set to 5, therefore the training and validation sets will be divided into five equal sets.
- "iterations": the number of iterations for the experiment is set to 20. It's a reasonable number and would provide the intendable result for the given dataset.
- compute_target: set to the project cluster to run the experiment.
- task: set to 'classification' since our target to predict whether the patient has diabetes or not.
- training_data: the loaded dataset for the project.
- label_column_name: set to the result/target colunm in the dataset 'Outcome' (0 or 1).
- enable_early_stopping: is enabled to terminate the experiment if the accuracy score is not showing improvement over time.
- featurization = is set to 'auto', it's an indicator of whether implementing a featurization step to preprocess/clean the dataset automatically or not. In our case, the preprocessing was applied for the numerical columns which normally involve treating missing values, cluster distance, the weight of evidence...etc.
- debug_log: errors will be logged into 'automl_errors.log'.


In [ ]:
# Automl settings 
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5,
    "iterations": 24
    
}

# Automl config 
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = 'classification',
                             training_data=dataset_updated,
                             label_column_name='Outcome',
                             enable_early_stopping= True,
                             featurization = 'auto',
                             debug_log = 'automl_errors.log',
                             **automl_settings
                            )



In [ ]:
# Submit experiment
remote_run = experiment.submit(automl_config, show_output=True)

## Run Details

The best model has resulted from the AutoML experiment from VotingEnsemble model. The Voting Ensemble model takes a majority vote of several algorithms which makes it surpass individual algorithms and minimize the bias. 

Use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion(show_output=True)

In [ ]:
remote_run

## Best Model

The cell below shows the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve and save best automl model
best_model, model = remote_run.get_output()
print(best_model)
print(model.steps)

In [ ]:
remote_run.get_metrics()

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

In the cell below, we have registered the model, created an inference config and deployed the model as a web service.

In [ ]:
# Registring the best model
model = best_model.register_model(model_name='automl-best-model',model_path='outputs/model.pkl')


In [ ]:
# from azureml.core.conda_dependencies import CondaDependencies

# # create environment
# environment = Environment(name="azure-env")
# conda_dep = CondaDependencies()

# # Nedded packages and scripts
# conda_dep.add_conda_package("pandas")
# conda_dep.add_conda_package("numpy")
# conda_dep.add_pip_package("scikit-learn")
# #conda_dep.add_conda_package("scikit-learn")
# conda_dep.add_pip_package("azureml-defaults")

# # Adding dependencies to the created environment
# environment.python.conda_dependencies=conda_dep

# Get automl environment with its dependencies
environment = Environment.get(ws, "AzureML-AutoML")

In [ ]:
inference_config = InferenceConfig(entry_script='scoring.py',
                                   environment=environment)
service_name = 'automl-deploy-1'
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config,
                       overwrite=True
                      )
service.wait_for_deployment(show_output=True)

scoring_uri = service.scoring_uri
print(scoring_uri)

In [ ]:
# Enable app insights
service.update(enable_app_insights=True)

In the cell below, we have sent a request to the web service to test it.

In [ ]:
import requests
import json

# URL for the web service, should be similar to:
#scoring_uri = 'http://0d469fee-33c0-4ce9-9dac-2767356110c9.southcentralus.azurecontainer.io/score'
# If the service is authenticated, set the key or token
#key = 'zuv0yY9prOPHFssuYSFWDJPhnVMxgJqG'
#two set of data to score, so we get two results back
data = {"data": [{"Pregnancies": 6, 
     "Glucose": 148, 
     "BloodPressure": 72, 
     "SkinThickness": 35, 
     "Insulin": 0, 
     "BMI": 33.5, 
     "DiabetesPedigreeFunction": 0.627, 
     "Age": 50},

    {"Pregnancies": 1, 
     "Glucose": 85, 
     "BloodPressure": 66, 
     "SkinThickness": 29, 
     "Insulin": 20, 
     "BMI": 26.5, 
     "DiabetesPedigreeFunction": 0.351, 
     "Age": 31},
      ]}
    
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())
print("Case 0: Not Diabetes, Case 1: Diabetes.")

In [ ]:

data = [{
           "Pregnancies": 6, 
             "Glucose": 148, 
             "BloodPressure": 72, 
             "SkinThickness": 35, 
             "Insulin": 0, 
             "BMI": 33.5, 
             "DiabetesPedigreeFunction": 0.627, 
             "Age": 50
           },
          {
            "Pregnancies": 1, 
             "Glucose": 85, 
             "BloodPressure": 66, 
             "SkinThickness": 29, 
             "Insulin": 20, 
             "BMI": 26.5, 
             "DiabetesPedigreeFunction": 0.351, 
             "Age": 31
          },
      ]
    
print(data)

In [ ]:
# test using service instance
input_data = json.dumps({
    'data': data
})

output = service.run(input_data)
output

Print the logs of the web service and delete the service

In [ ]:
logs = service.get_logs()
logs

In [ ]:
service.delete()